In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
import torch
import torch.nn as nn
import torch.optim as optim

from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

np.set_printoptions(precision=4, linewidth=80, threshold=10)
sns.set_style('whitegrid')
torch.set_printoptions(precision=4, linewidth=80, threshold=10)

device = "mps" if torch.backends.mps.is_available else "cpu"
print(f"Current device is {device}")

## 1.

In [ ]:
data = pd.read_excel('./Data/data.xlsx')
data.index = data.Date
data.drop(columns=['Date'], inplace=True)

In [ ]:
# Загрузим гиперпараметры
%store -r h
%store -r sigma
%store -r delta
%store -r n

clear_output()

In [ ]:
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

y_train, y_test, X_train, X_test = temporal_train_test_split(
    data['target'], data.iloc[:, :-1], test_size=50
)

X_train, Z_train = X_train.iloc[:, :-1], X_train.values[:, -1].reshape(-1, 1)
# X_train_scaled = X_scaler.fit_transform(X_train)
X_train_scaled = X_train.values

X_test, Z_test = X_test.iloc[:, :-1], X_test.values[:, -1].reshape(-1, 1)
# X_test_scaled = X_scaler.transform(X_test)
X_test_scaled = X_test.values

y_train_scaled = y_train.values.reshape(-1, 1)
y_test_scaled = y_test.values.reshape(-1, 1)

# y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
# y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))

In [ ]:
X_train_tensor = torch.tensor(X_train_scaled.reshape((-1, h, 1)).astype('float32'))
y_train_tensor = torch.tensor(y_train_scaled.astype('float32'))
Z_train_tensor = torch.tensor(Z_train.astype('float32'))

X_test_tensor = torch.tensor(X_test_scaled.reshape((-1, h, 1)).astype('float32'))
y_test_tensor = torch.tensor(y_test_scaled.astype('float32'))
Z_test_tensor = torch.tensor(Z_test.astype('float32'))

X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape, Z_train_tensor.shape, Z_test_tensor.shape

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor, Z_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor, Z_test_tensor)

In [ ]:
%store -r batch_size

# batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch, z_batch = batch[0].to(device), batch[1].to(device), batch[2].to(device)
    print(x_batch.shape, y_batch.shape, z_batch.shape)
    break

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTM(1, 4, 1)
model.to(device)
model

In [ ]:
# class LSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, num_stacked_layers):
#         super().__init__()
#         self.hidden_size = hidden_size
#         self.num_stacked_layers = num_stacked_layers
# 
#         # Первый LSTM слой
#         self.lstm1 = nn.LSTM(input_size, hidden_size, num_stacked_layers, batch_first=True)
# 
#         # Второй LSTM слой
#         self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_stacked_layers, batch_first=True)
# 
#         # Полносвязанный слой для выхода
#         self.fc = nn.Linear(hidden_size, 1)
# 
#     def forward(self, x):
#         batch_size = x.size(0)
#         h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(x.device)
#         c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(x.device)
# 
#         # Проход через первый LSTM слой
#         out, _ = self.lstm1(x, (h0, c0))
# 
#         # Проход через второй LSTM слой
#         out, _ = self.lstm2(out, (h0, c0))
# 
#         # Проход через полносвязанный слой
#         out = self.fc(out[:, -1, :])
# 
#         return out
# 
# model = LSTM(1, 4, 1)
# model.to(device)
# model

In [ ]:
def CustomMSE(f_curr, f_prev, y_curr, Z_curr, sigma):
    predictions = (f_curr - f_prev) * np.sqrt(delta) + sigma * Z_curr
    mse = torch.mean((y_curr - predictions)**2)
    return mse

In [ ]:
def CustomMAE(f_curr, f_prev, y_curr, Z_curr, sigma):
    predictions = (f_curr - f_prev) * np.sqrt(delta) + sigma * Z_curr
    mse = torch.mean(torch.abs(y_curr - predictions))
    return mse

In [ ]:
learning_rate = 0.001
num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loss_history, test_loss_history = [], []
best_test_loss = float('inf')
best_epoch = 0

for epoch in tqdm(range(1, num_epochs + 1)):
    clear_output(wait=True)
    torch.manual_seed(66)
    
    model.train()
    curr_train_loss = 0
    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch, z_batch = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        
        f_all = model(x_batch)
        f_curr, f_prev = f_all[1:], f_all[:-1]
        
        loss = CustomMSE(f_curr, f_prev, y_batch[1:], z_batch[1:], sigma)
        # loss = CustomMAE(f_curr, f_prev, y_batch[1:], z_batch[1:], sigma)
        curr_train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    train_loss_history.append(curr_train_loss)
        
    model.eval()
    curr_test_loss = 0
    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch, z_batch = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        
        with torch.no_grad():
            f_all = model(x_batch)
            f_curr, f_prev = f_all[1:], f_all[:-1]
            loss = CustomMSE(f_curr, f_prev, y_batch[1:], z_batch[1:], sigma)
            # loss = CustomMAE(f_curr, f_prev, y_batch[1:], z_batch[1:], sigma)
            curr_test_loss += loss.item()
    
    # # Здесь сохраним лучшую модель
    # if curr_test_loss < best_test_loss:
    #     best_test_loss = curr_test_loss
    #     model_filename = f'best_model_{sigma}.pth'
    #     torch.save(model, model_filename)
    #     best_epoch = epoch
        
            
    test_loss_history.append(curr_test_loss)
    
    print("Current status:")
    print(f"Curren h: {h}")
    print(f"Curren sigma: {sigma}")
    print(f"Curren batch_size: {batch_size}")
    
    print(f"Лучшая эпоха: {best_epoch}")
    # Выведем графики
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(24, 10))
    axes = axes.flatten()
    
    fig.suptitle(f"Results with sigma = {sigma}")

    sns.lineplot(x=range(1, epoch + 1), y=train_loss_history, label='train', color='blue', ax=axes[0])
    axes[0].set_xlabel('epoch')
    axes[0].set_ylabel('loss')
    
    sns.lineplot(x=range(1, epoch + 1), y=test_loss_history, label='test', color='orange', ax=axes[1])
    axes[1].set_xlabel('epoch')
    axes[1].set_ylabel('loss')
    
    
    with torch.no_grad():
        f_all = model(X_train_tensor.to(device)).to('cpu').numpy()
        f_curr, f_prev = f_all[1:], f_all[:-1]
     
    y_pred_scaled = (f_curr - f_prev) * np.sqrt(delta) + sigma * Z_train[1:]
     
    # y_pred_train = y_scaler.inverse_transform(y_pred_scaled)
    y_pred_train = y_pred_scaled
    
    sns.lineplot(x=range(1, y_train.values[1:].shape[0] + 1), y=y_train.values[1:], label="real train", ax=axes[2])
    sns.lineplot(x=range(1, y_pred_train.flatten().shape[0] + 1), y=y_pred_train.flatten(), label="predicted train", ax=axes[2])
    
    with torch.no_grad():
        f_all = model(X_test_tensor.to(device)).to('cpu').numpy()
        f_curr, f_prev = f_all[1:], f_all[:-1]
     
    y_pred_scaled = (f_curr - f_prev) * np.sqrt(delta) + sigma * Z_test[1:]
     
    # y_pred_test = y_scaler.inverse_transform(y_pred_scaled)
    y_pred_test = y_pred_scaled
        
    plot_series(
        y_test[1:], pd.Series(y_pred_test.flatten(), index=y_test[1:].index),
        labels=['test', 'pred'],
        ax=axes[3]
    )
    
    axes[3].set_xticks(range(1, len(y_test[1:].index) + 1, 10))
    axes[3].set_xticklabels(y_test[1::10].index.date, rotation=10)
    
    
    plt.legend()
    plt.show()

In [ ]:
# clear_output()
# 
# with torch.no_grad():
#     f_all = model(X_train_tensor.to(device)).to('cpu').numpy()
#     f_curr, f_prev = f_all[1:], f_all[:-1]
#     
#     y_pred_scaled = (f_curr - f_prev) * np.sqrt(delta) + sigma * Z_train[1:]
#     
# # y_pred_train = y_scaler.inverse_transform(y_pred_scaled)
# y_pred_train = y_pred_scaled
# 
# plt.figure(figsize=(15, 7))
# plt.plot(y_train.values[1:], label="real train")
# plt.plot(y_pred_train, label="predicted train")
# plt.ylabel("Delta")
# plt.legend()
# plt.show()

In [ ]:
# with torch.no_grad():
#     f_all = model(X_test_tensor.to(device)).to('cpu').numpy()
#     f_curr, f_prev = f_all[1:], f_all[:-1]
#     
#     y_pred_scaled = (f_curr - f_prev) * np.sqrt(delta) + sigma * Z_test[1:]
#     
# # y_pred_test = y_scaler.inverse_transform(y_pred_scaled)
# y_pred_test = y_pred_scaled
# 
# plt.figure(figsize=(15, 7))
# plt.plot(y_test.values[1:], label="real train")
# plt.plot(y_pred_test, label="predicted train")
# plt.ylabel("Delta")
# plt.savefig(f"")
# plt.legend()
# plt.show()

In [ ]:
# time.sleep(3)
clear_output()

with torch.no_grad():
    f_all = model(X_test_tensor.to(device)).to('cpu').numpy()
    f_curr, f_prev = f_all[1:], f_all[:-1]
    
    y_pred_scaled = (f_curr - f_prev) * np.sqrt(delta) + sigma * Z_test[1:]
    
# y_pred_test = y_scaler.inverse_transform(y_pred_scaled)
y_pred_test = y_pred_scaled
plot_series(
    y_test[1:], pd.Series(y_pred_test.flatten(), index=y_test[1:].index),
    labels=['test', 'pred']
)

plt.xlabel("Date")
plt.xticks(rotation=10)
plt.title(f"Predictions on the test sample with sigma = {sigma}")
plt.ylim(-0.15, 0.15)
plt.savefig(f"./Images/test_pred_sigma:{sigma}.png")
plt.show()
time.sleep(3)

### Лучшая модель

In [ ]:
# best_model = torch.load("best_noisy.pth", map_location=torch.device('mps'))

In [ ]:
# with torch.no_grad():
#     f_all = best_model(X_train_tensor.to(device)).to('cpu').numpy()
#     f_curr, f_prev = f_all[1:], f_all[:-1]
#     
#     y_pred_scaled = (f_curr - f_prev) * np.sqrt(delta) + sigma * Z_train[1:]
#     
# # y_pred_train = y_scaler.inverse_transform(y_pred_scaled)
# y_pred_train = y_pred_scaled
# 
# plt.figure(figsize=(15, 7))
# plt.plot(y_train.values[1:], label="real train")
# plt.plot(y_pred_train, label="predicted train")
# plt.ylabel("Delta")
# plt.legend()
# plt.show()

In [ ]:
# with torch.no_grad():
#     f_all = best_model(X_test_tensor.to(device)).to('cpu').numpy()
#     f_curr, f_prev = f_all[1:], f_all[:-1]
#     
#     y_pred_scaled = (f_curr - f_prev) * np.sqrt(delta) + sigma * Z_test[1:]
#     
# # y_pred_test = y_scaler.inverse_transform(y_pred_scaled)
# y_pred_test = y_pred_scaled
# 
# # plt.figure(figsize=(15, 7))
# # plt.plot(y_test.values[1:], label="real train")
# # plt.plot(y_pred_test, label="predicted train")
# # plt.ylabel("Delta")
# # plt.legend()
# # plt.show()

In [ ]:
# plot_series(
#     y_test[1:], pd.Series(y_pred_test.flatten(), index=y_test[1:].index),
#     labels=['test', 'pred']
# )
# 
# plt.xticks(rotation=10)
# plt.show()